In [1]:
from splinter import Browser
from bs4 import BeautifulSoup as souper
import pandas as pd
import re
import time

### Setup Functions to Help With Scraping
-----------------------------------------

In [30]:
def tableScrape(table, formtype, tabletype):
    '''Function to scrape SEC tables for Meta, Inc. Expense Reports

    Args: 
            table: html text to scrape (string)
            formtype: specify the form type (string)
            tabletype: table from report being scraped (string)

    Returns:
            ret_dict: dictionary containing relevant table data (dictionary obj)
    '''
    
    # Select index values for rows with a dollar sign vs rows without
    dollar, no_dollar, cell_indexes = specifyReportType(formtype, tabletype)   
    # Get relevant rows from table
    ret_dict = {}
    for tr in table.find_all("tr"):
        tds = tr.find_all("td")
        if type(tds) != "bool": # Filters boolean rows which wouldn't have a len property
            ltds = len(tds)
            if (ltds == no_dollar) | (ltds == dollar): # rows with numeric values have specific number of tags
                for i in range(len(tds)):
                    try: # converting values to integers only works on cells with numeric values
                        if i == 0:
                            title = tds[i].text # Row label always in first row in these tables
                            ret_dict[title] = []
                        if i in cell_indexes[ltds]:
                            amount = tds[i].text
                            amount = amount.split("\xa0")[0]
                            if amount[0] == "(":
                                amount = amount[1:-1]
                            amounts = amount.split(",")
                            amount = int("".join(amounts))
                            ret_dict[title] += [amount]
                    except: # skip rows that don't contain relevant numeric data
                        try:
                            del ret_dict[title]
                        except:
                            continue
                        continue
    
    return ret_dict

def getShareRepurchases(table):
    '''Function specifically to get share repurchase data

    Args:
            table: html text to scrape (string)

    Returns:
            data: dictionary containing share repurchase data (dictionary obj)
    '''
    
    # Get Share Repurchase Data
    trs = table.find_all("tr")
    data = {"Share repurchases": []}
    for tr in trs:
        tds = tr.find_all("td")
        if tds[0].text == "Share repurchases": # isolates only Share repurchase row
            shares = [int(tds[1].text[1:-1])]
            data["Share repurchases"] += shares
    return data

def listIndexes(num_cells, rtype):
    '''Generates list of indexes to scrape for each report type

    Args:
        num_cells: number of cells in row (int)
        rtype: choice for columns with dollar signs and without (string)
    Returns:
        ret_list: list of indexes for cells with numeric data
    '''
    ret_list = [] 
    counter = num_cells - 2
    if rtype == "dollar":
        while counter > 0:
            ret_list.append(counter)
            counter -= 4
    if rtype == "no_dollar":
        while counter > 0:
            ret_list.append(counter)
            counter -= 3
    return ret_list

def specifyReportType(formtype, tabletype):
    '''Function to get values to use for proper report scraping

    Args:
        formtype: time interval of the form being scraped (string)
        tabletype: the name of the table being scraped (string)

    Returns:
        dollar: number of cells for rows with dollar signs (int)
        no_dollar: number of cells for rows with no dollar signs (int)
        indexes: dictionary of values to reference for tableScrape (dictionary obj)
    '''
    if tabletype == "CONSOLIDATED BALANCE SHEETS":
        dollar = 8
        no_dollar = 6
    if formtype == "10-Q":
        if tabletype == "CONSOLIDATED STATEMENTS OF INCOME":
            dollar = 16
            no_dollar = 12
    if formtype == "10-K":
        if tabletype == "CONSOLIDATED STATEMENTS OF INCOME":
            dollar = 12
            no_dollar = 9
    d = listIndexes(dollar, "dollar")
    nd = listIndexes(no_dollar, "no_dollar")
    indexes = {dollar: d, no_dollar: nd}
    return dollar, no_dollar, indexes

def pagePrep():
    '''Navigates to SEC Report Filings Page for Meta, Inc. and Lists Reports'''
    # Navigate to main tab and close all others
    try:
        browser.windows.current = browser.windows[0]
        window = browser.windows[0]
        window.close_others()
    except:
        None
    
    # Navigate to EDGAR landing page for specified company
    base_url = "https://www.sec.gov/edgar/browse/?CIK=1326801&owner=exclude" # SEC reports for Meta, Inc.
    browser.visit(base_url)
    parent_card_div = browser.find_by_id('filingsStart')
    all_card_divs = parent_card_div.find_by_css("div[class='card']")
    third_div = all_card_divs[3] # div card for quarterly and annual reports
    third_div.click() # activate javascript to open buttons
    third_div_child = third_div.find_by_tag("div").first
    third_div_child.find_by_tag("button").first.click() # button action to list reports
    time.sleep(1) # timeout so that webpage can load before further actions taken
    return

def pageVisit(fdate, fdescription):
    '''Function to get to specified report page'''
    pagePrep()
    browser.links.find_by_text(fdate).first.click()
    time.sleep(1)
    browser.links.find_by_text(fdescription).first.click()
    browser.windows.current = browser.windows[1] # navigate to newly opened window

    # Change XML Report into basic HTML page
    clicked_url = browser.url
    url_parts = clicked_url.split("/ix?doc=")
    clean_url = "".join(url_parts)
    browser.visit(clean_url)
    return

def findEndBracket(text):
    '''Function to find index value of End Bracket in text

    Args:
        text: text string to search (string)
    Returns:
        i: index value of text (int)
    '''
    for i, data in enumerate(text):
        if data == "]":
            return i
    return None

def getReportsList(title):
    '''Build DataFrame of the list of reports on the first page of the Reports list

    Args:
        title: company name being used (string)
    Returns:
        form_frame: dataframe of reprots on first page of list (DataFrame Object)
    '''
    # Build DataFrame
    pagePrep()
    html = browser.html
    souped = souper(html, "html.parser")
    header_div = souped.find("div", class_="dataTables_scrollHead")
    header_table = header_div.find("table")
    headers = [h.text for h in header_table.find_all("th")]
    data_table = souped.find("table", id="filingsTable")
    body = data_table.find("tbody")
    data = [[td.text for td in tr.find_all("td")] for tr in body.find_all("tr")]
    form_frame = pd.DataFrame(data=data, columns=headers)
    
    # Clean Columns and export/return cleaned DataFrame
    extra_text = "View all with same reporting date"
    form_frame["Form description"] = [data[:1+findEndBracket(data)] for data in form_frame["Form description"].values]
    form_frame["Reporting date"] = [data[: -len(extra_text)] for data in form_frame["Reporting date"].values]
    form_frame.to_csv(f'./Resources/EDGAR_landing_page_{title}.csv')
    return form_frame

def getTable(ttext):
    '''Function to return all tags from specified table

    Args:
        ttext: the text needed to find specific table (string)
    Returns:
        table: all tags/text from specifed table (string)
    '''
    find_string = souped.find("span", string=re.compile(ttext))
    if find_string == None:
        print("Table Text Not Found")
        return
    table_div = find_string.parent
    table = table_div.table
    while table == None:
        table_div = table_div.next_sibling
        table = table_div.table
    return table

def getIndexLables(table, formtype, tabletype, date):
    '''Get Index Labels for Each Kind of Table

    Args:
        table: table data (string)
        formtype: time period of financial statement (string)
        tabletype: the title of the table being scraped (string)
        date: Reproting date (date)
    Returns:
        indexes: index list to use when creating DataFrame (list)
    '''
    years = []
    indexes = []
    month3 = f"3 Month Ended {date}"
    month6 = f"6 Month Ended {date}"
    trs = table.find_all("tr")
    if formtype == "10-Q":
        if tabletype == "CONSOLIDATED STATEMENTS OF INCOME":
            period = [month3, month3, month6, month6]
            year_tags = trs[2]
            year_tds = year_tags.find_all("td")
            cell_indexes = [1,3,5,7]
            for i in cell_indexes:
                years.append(year_tds[i].text)
            arr_i = [period, years]
            indexes = pd.MultiIndex.from_tuples(list(zip(*arr_i)), names=["period", "years"])
        if tabletype == "CONSOLIDATED BALANCE SHEETS":
            date_tags = trs[1]
            date_tds = date_tags.find_all("td")
            cell_indexes = [1,3]
            for i in cell_indexes:
                indexes.append(date_tds[i].text)
        if tabletype == "CONSOLIDATED STATEMENTS OF STOCKHOLDERS' EQUITY":
            indexes = [month3, month6]
        return indexes 

## Gather Necessary Data
------------------------

In [4]:
# Setup Scraping Browser
browser = Browser("chrome")

In [5]:
reports_df = getReportsList("meta")
reports_df

,Form type,Form description,Filing date,Reporting date
0,10-Q,Quarterly report [Sections 13 or 15(d)],2024-08-01,2024-06-30
1,10-Q,Quarterly report [Sections 13 or 15(d)],2024-04-25,2024-03-31
2,10-K,"Annual report [Section 13 and 15(d), not S-K I...",2024-02-02,2023-12-31
3,10-Q,Quarterly report [Sections 13 or 15(d)],2023-10-26,2023-09-30
4,10-Q,Quarterly report [Sections 13 or 15(d)],2023-07-27,2023-06-30
5,10-Q,Quarterly report [Sections 13 or 15(d)],2023-04-27,2023-03-31
6,10-K,"Annual report [Section 13 and 15(d), not S-K I...",2023-02-02,2022-12-31
7,10-Q,Quarterly report [Sections 13 or 15(d)],2022-10-27,2022-09-30
8,10-Q,Quarterly report [Sections 13 or 15(d)],2022-07-28,2022-06-30
9,10-Q,Quarterly report [Sections 13 or 15(d)],2022-04-28,2022-03-31


In [26]:
quarterly = reports_df[reports_df["Form type"] == "10-Q"].reset_index(drop=True)
table_texts = ["CONSOLIDATED BALANCE SHEETS",
               "CONSOLIDATED STATEMENTS OF INCOME",
              "CONSOLIDATED STATEMENTS OF STOCKHOLDERS' EQUITY"]

In [32]:
df_list = []
for i in range(len(quarterly)):
    formtype = quarterly["Form type"][i]
    form_desc = quarterly["Form description"][i]
    filing_date = quarterly["Reporting date"][i]
    pageVisit(filing_date, form_desc)
    html = browser.html
    souped = souper(html, "html.parser")
    for j, text in enumerate(table_texts):
        table = getTable(text)
        if j != 2:
            data = tableScrape(table, formtype, text)
        else:
            data = getShareRepurchases(table)
        date = quarterly["Reporting date"][i]
        indexes = getIndexLables(table, formtype, text, date)
        df = pd.DataFrame(data, index=indexes)
        df_list.append(df)

In [39]:
test_subset = df_list[::3]
test_subset[6]

,Cash and cash equivalents,Marketable securities,"Accounts receivable, net",Prepaid expenses and other current assets,Total current assets,Non-marketable equity securities,"Property and equipment, net",Operating lease right-of-use assets,"Intangible assets, net",Goodwill,...,Deferred revenue and deposits,Total current liabilities,"Operating lease liabilities, non-current",Other liabilities,Total liabilities,Additional paid-in capital,Accumulated other comprehensive loss,Retained earnings,Total stockholders' equity,Total liabilities and stockholders' equity
"June 30,2022",12681,27808,11525,3973,55987,6536,67588,14130,965,20229,...,532,22217,14792,7003,44012,59929,3411,69249,125767,169779
"December 31,2021",16601,31397,14039,4629,66666,6775,57809,12155,634,19197,...,561,21135,12746,7227,41108,55811,693,69761,124879,165987
